# Data Set Up

In [1]:
!pip install pyathena

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install awswrangler

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 211 kB 6.9 MB/s            
     |████████████████████████████████| 207 kB 100.9 MB/s            
     |████████████████████████████████| 43 kB 4.8 MB/s             
     |████████████████████████████████| 94 kB 9.1 MB/s              
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy import intersect1d
from datetime import datetime
from pyathena import connect
import boto3
import awswrangler as wr

Matplotlib is building the font cache; this may take a moment.


In [4]:
from ethereum import (
    timestamp_to_datetime,
    EthereumData
)
from utils import (
    latest,
    lead_lag,
    add_latest_avail_block,
    lagged_block_data
)

## Blocks

In [5]:
blocks_df = wr.s3.read_csv(path="s3://sagemaker-w210-eth/raw_data/infuria/blocks")

In [6]:
blocks_df.head()

,number,hash,parent_hash,nonce,sha3_uncles,logs_bloom,transactions_root,state_root,receipts_root,miner,difficulty,total_difficulty,size,extra_data,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas
0,12962018,0x93288d45fd0daac2605174fc6d14d27a1f18ed6d72ae...,0x9c42cc01eb5564709c476eff08065a67fc8e479e996c...,0xf1828e43fadfc703,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x60af99ca65d7000f80980c1cef011a3b585619005110...,0xa6d240b3a7409a4f197e347d5dbed9159802e4787b82...,0x6459e84f5309663bb565625ed662809c0233889631c8...,0x0cadca37374ac1442ab84af32b4d1dc0e90837b1fc71...,0x829bd824b016326a401d083b33d092293333a830,7604452341876898,28471525393295665706633,55433,0xe4b883e5bda9e7a59ee4bb99e9b1bc020321,15000000,14994306,1628126126,160,NaN
1,12962019,0xeb13f4f4bc463d6ff438df7ea04074a064ca95b474cc...,0x93288d45fd0daac2605174fc6d14d27a1f18ed6d72ae...,0xa4bfd2e7099b4a6b,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x532383a629274b7190ccc2d0aa02986d54e296d49823...,0x9b9701c0c94d550770390ec857e85e650874ec32cd79...,0xb75212595e6aedf34bae4f08d1dddd3a34a933f0cb09...,0x95cda15e5b562751ce34c3760b5ebedae43fc60fe9d5...,0x8595dd9e0438640b5e1254f9df579ac12a86865f,7604589780830370,28471532997885446537003,79403,0x657a696c2e6d65,14985353,14969945,1628126136,217,NaN
2,12962020,0x50ded5bb455b2bc00392dd36dcfff3d3019eb9517478...,0xeb13f4f4bc463d6ff438df7ea04074a064ca95b474cc...,0xe21691d992c44366,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x30f7606e59608213d0e6f610b860da97c90291082a18...,0x6e39ad0229b49c1c0793d4ac9f84ee3cfafde05f31ca...,0xb533fcc03912aad33abe748bd627bd1ca1233284a462...,0xdcabba7c1257c64d0ed6d845048727a3291aed33f358...,0x7f101fe45e6649a6fb8f3f8b43ed03d353f2b90c,7601014041179921,28471540598899487716924,69949,0x466c6578706f6f6c2f53302f55532d45617374202d20...,14970720,14964505,1628126157,155,NaN
3,12962021,0xc3e9e7c41845ae59c14e7b278ab9a2d4c56368761d3c...,0x50ded5bb455b2bc00392dd36dcfff3d3019eb9517478...,0x130800068dfdf6ed,0x34db8d1d8cd5772d238ecfa7e41114456c3f188235b9...,0xe0b1082ad3da6c93ca8e20eda8619e514f6651111166...,0x463fa688671b03a26f1812b161dab03b7ba55d4f2336...,0xd9e98811a4b7d7a22912ded685ff1a1537eda9729509...,0xe654f88bcd7177c2e5457910a848513dd475920897db...,0x09ab1303d3ccaf5f018cd511146b07a240c70294,7601151480133393,28471548200050967850317,92704,0x6d696e6572616c6c2e696f,14956102,14944125,1628126167,195,NaN
4,12962022,0x662aac52ee02b41ac3f3de32382eff0c5d1a4eeb31d2...,0xc3e9e7c41845ae59c14e7b278ab9a2d4c56368761d3c...,0x979e4f65cdc6a89d,0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a...,0x76a1f1b641489b8d5090e6dfa2781725a2c61333f898...,0x8371509cd79d207d83603fec4397472abc9deb1f8c82...,0xacc983062bb1a54aded29c48652d5def40c47987f424...,0xd6924c31610c313db53b63a03554b8e124d1b6a3195e...,0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c,7597577419340707,28471555797628387191024,65855,0xd883010a06846765746888676f312e31362e36856c69...,14963388,14943452,1628126200,178,NaN


In [7]:
print("Blocks shape:", blocks_df.shape)
print("Minimum block number:", blocks_df.number.min())
print("Maximum block number:", blocks_df.number.max())

Blocks shape: (566141, 19)
Minimum block number: 12961718
Maximum block number: 13527858


In [8]:
cols = [
    'number', 'difficulty', 'total_difficulty', 'size', 'gas_limit', 
    'gas_used', 'timestamp', 'transaction_count', 'base_fee_per_gas'
]

blocks_df = blocks_df[cols]


In [9]:
print("# of unique blocks:", len(pd.unique(blocks_df.number)))

# of unique blocks: 566141


In [10]:
blocks_df.head()

,number,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas
0,12962018,7604452341876898,28471525393295665706633,55433,15000000,14994306,1628126126,160,NaN
1,12962019,7604589780830370,28471532997885446537003,79403,14985353,14969945,1628126136,217,NaN
2,12962020,7601014041179921,28471540598899487716924,69949,14970720,14964505,1628126157,155,NaN
3,12962021,7601151480133393,28471548200050967850317,92704,14956102,14944125,1628126167,195,NaN
4,12962022,7597577419340707,28471555797628387191024,65855,14963388,14943452,1628126200,178,NaN


In [11]:
# set index and add readable timestamp
blocks_df = blocks_df.set_index('number')
blocks_df['timestamp_2'] = pd.to_datetime(blocks_df['timestamp'], unit='s')

In [12]:
blocks_df = blocks_df.sort_index()

In [13]:
blocks_df.head()

,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,timestamp_2
number,,,,,,,,,
12961718,7774193366499997,28469210465037599682985,73653,14970676,14957821,1628121604,152,NaN,2021-08-05 00:00:04
12961719,7778126798308205,28469218243164397991190,103790,14985294,14975998,1628121607,235,NaN,2021-08-05 00:00:07
12961720,7766870496834469,28469226010034894825659,101080,14999927,14981552,1628121647,196,NaN,2021-08-05 00:00:47
12961721,7770800353022723,28469233780835247848382,68027,15000000,14987640,1628121663,194,NaN,2021-08-05 00:01:03
12961722,7767143455866321,28469241547978703714703,543,15000000,0,1628121685,0,NaN,2021-08-05 00:01:25


In [14]:
blocks_df['base_fee_per_gas'].describe()

count    5.628590e+05
mean     8.389799e+10
std      7.564743e+10
min      1.000000e+09
25%      4.832533e+10
50%      6.923107e+10
75%      9.901271e+10
max      3.941920e+12
Name: base_fee_per_gas, dtype: float64

#### Create percent changes in variables

In [15]:
cols = ['difficulty', 'total_difficulty', 'size', 'gas_limit', 'base_fee_per_gas']
for c in cols:
    print(c, blocks_df[c].dtype)

difficulty int64
total_difficulty object
size int64
gas_limit int64
base_fee_per_gas float64


In [16]:
# Change difficulty to float to calculate change in difficulty
blocks_df['total_difficulty'] = blocks_df['total_difficulty'].astype(float)

In [17]:
cols = ['difficulty', 'total_difficulty', 'size', 'gas_limit', 'base_fee_per_gas']
for col in cols:
    blocks_df[col+'_pct_chg'] = blocks_df[col]/blocks_df[col].shift(1)-1

In [18]:
blocks_df.tail()

,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,timestamp_2,difficulty_pct_chg,total_difficulty_pct_chg,size_pct_chg,gas_limit_pct_chg,base_fee_per_gas_pct_chg
number,,,,,,,,,,,,,,
13527854,10142061783382444,3.349723e+22,59466,30029295,14081332,1635724692,185,1.721710e+11,2021-10-31 23:58:12,0.000495,3.027732e-07,3.496484,0.000976,-0.098995
13527855,10142130502859180,3.349724e+22,113946,30058619,30037769,1635724708,407,1.708332e+11,2021-10-31 23:58:28,0.000007,3.027752e-07,0.916154,0.000977,-0.007770
13527856,10127342585857119,3.349725e+22,101346,30029266,20028596,1635724748,186,1.921577e+11,2021-10-31 23:59:08,-0.001458,3.023336e-07,-0.110579,-0.000977,0.124827
13527857,10127411305333855,3.349726e+22,12068,30000000,2550891,1635724758,30,2.001788e+11,2021-10-31 23:59:18,0.000007,3.023356e-07,-0.880923,-0.000975,0.041742
13527858,10132425049862023,3.349727e+22,132704,29980831,29719237,1635724760,408,1.794118e+11,2021-10-31 23:59:20,0.000495,3.024852e-07,9.996354,-0.000639,-0.103743


#### Calculate lagged variables from blocks to join to PIT

In [19]:
cols = ['difficulty', 'total_difficulty', 'size', 'gas_limit', 'base_fee_per_gas']
for col in cols:
    # Last 5 blocks
    blocks_df[col+'_pct_chg_last_5'] = blocks_df[col]/blocks_df[col].shift(5)-1
    # 25 blocks ago to 5 blocks ago percentage changes
    blocks_df[col+'_pct_chg_last_25_to_5'] = blocks_df[col].shift(5)/blocks_df[col].shift(25)-1
    # 50 blocks ago to 5 blocks ago percentage changes
    blocks_df[col+'_pct_chg_last_50_to_5'] = blocks_df[col].shift(5)/blocks_df[col].shift(50)-1
    # 100 blocks ago to 5 blocks ago percentage changes
    blocks_df[col+'_pct_chg_last_100_to_5'] = blocks_df[col].shift(5)/blocks_df[col].shift(100)-1

In [20]:
blocks_df.head()

,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,timestamp_2,difficulty_pct_chg,...,size_pct_chg_last_50_to_5,size_pct_chg_last_100_to_5,gas_limit_pct_chg_last_5,gas_limit_pct_chg_last_25_to_5,gas_limit_pct_chg_last_50_to_5,gas_limit_pct_chg_last_100_to_5,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_25_to_5,base_fee_per_gas_pct_chg_last_50_to_5,base_fee_per_gas_pct_chg_last_100_to_5
number,,,,,,,,,,,,,,,,,,,,,
12961718,7774193366499997,2.846921e+22,73653,14970676,14957821,1628121604,152,NaN,2021-08-05 00:00:04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961719,7778126798308205,2.846922e+22,103790,14985294,14975998,1628121607,235,NaN,2021-08-05 00:00:07,0.000506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961720,7766870496834469,2.846923e+22,101080,14999927,14981552,1628121647,196,NaN,2021-08-05 00:00:47,-0.001447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961721,7770800353022723,2.846923e+22,68027,15000000,14987640,1628121663,194,NaN,2021-08-05 00:01:03,0.000506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961722,7767143455866321,2.846924e+22,543,15000000,0,1628121685,0,NaN,2021-08-05 00:01:25,-0.000471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
blocks_df.describe()

,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,difficulty_pct_chg,total_difficulty_pct_chg,...,size_pct_chg_last_50_to_5,size_pct_chg_last_100_to_5,gas_limit_pct_chg_last_5,gas_limit_pct_chg_last_25_to_5,gas_limit_pct_chg_last_50_to_5,gas_limit_pct_chg_last_100_to_5,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_25_to_5,base_fee_per_gas_pct_chg_last_50_to_5,base_fee_per_gas_pct_chg_last_100_to_5
count,5.661410e+05,5.661410e+05,5.661410e+05,5.661410e+05,5.661410e+05,5.661410e+05,566141.000000,5.628590e+05,5.661400e+05,5.661400e+05,...,566091.000000,566041.000000,566136.000000,566116.000000,566091.000000,566041.000000,562854.000000,562834.000000,562809.000000,562759.000000
mean,8.881305e+15,3.086043e+22,7.718583e+04,2.992329e+07,1.544216e+07,1.631910e+09,188.815684,8.389799e+10,7.155172e-07,2.872804e-07,...,4.088010,4.129498,0.000010,0.000036,0.000080,0.000167,0.012018,0.031181,0.061357,0.086395
std,7.772472e+14,1.456622e+21,5.759466e+04,1.146282e+06,1.029474e+07,2.193156e+06,142.875043,7.564743e+10,7.033183e-04,1.283190e-08,...,23.283212,23.399238,0.003215,0.006055,0.008896,0.012699,0.160737,0.333681,0.758804,0.915449
min,7.213602e+15,2.846921e+22,5.200000e+02,1.489037e+07,0.000000e+00,1.628122e+09,0.000000,1.000000e+09,-8.297229e-03,2.519518e-07,...,-0.999368,-0.999269,-0.004873,-0.015512,-0.024138,-0.031755,-0.470438,-0.863385,-0.953787,-0.972170
25%,8.224747e+15,2.957782e+22,2.827500e+04,3.000000e+07,6.122355e+06,1.630011e+09,74.000000,4.832533e+10,-4.816755e-04,2.784650e-07,...,-0.597942,-0.598376,-0.000976,-0.000976,-0.000976,-0.000976,-0.100389,-0.131083,-0.149842,-0.167211
50%,8.964495e+15,3.080368e+22,6.619700e+04,3.000000e+07,1.396269e+07,1.631902e+09,161.000000,6.923107e+10,6.689213e-06,2.894665e-07,...,-0.001240,-0.003027,0.000000,0.000000,0.000000,0.000000,-0.011156,-0.008042,-0.008954,-0.008530
75%,9.495806e+15,3.211024e+22,1.189830e+05,3.002930e+07,2.633653e+07,1.633807e+09,283.000000,9.901271e+10,4.892359e-04,2.967394e-07,...,1.486775,1.486212,0.000975,0.000976,0.000976,0.000976,0.099221,0.137729,0.160197,0.183855
max,1.066084e+16,3.349727e+22,1.457148e+06,3.038299e+07,3.029293e+07,1.635725e+09,1431.000000,3.941920e+12,9.949027e-04,3.193828e-07,...,855.144424,889.976364,0.999015,0.996091,1.003898,0.998041,0.801837,9.533298,50.936090,58.384491


## Transactions

In [ ]:
# Old way

# transactions_df_1 = pd.read_csv(r'data/transactions_small/12961718_13136426.csv')
# transactions_df_2 = pd.read_csv(r'data/transactions_small/13136427_13330089.csv')
# transactions_df_3 = pd.read_csv(r'data/transactions_small/13330090_13527858.csv')

# print("transactions_df_1 shape:", transactions_df_1.shape)
# print("transactions_df_2 shape:", transactions_df_2.shape)
# print("transactions_df_3 shape:", transactions_df_3.shape)

# transactions_df = pd.concat([transactions_df_1, transactions_df_2, transactions_df_3], ignore_index=True)
# transactions_df.shape

# transactions_df['block_timestamp_2'] = pd.to_datetime(transactions_df['block_timestamp'], unit='s')

# cols = [
#     'transaction_id', 'block_number', 'transaction_index', 'value', 
#     'gas', 'gas_price', 'block_timestamp', 'max_fee_per_gas', 'max_priority_fee_per_gas',
#     'transaction_type'
# ]
# transactions_df = transactions_df[cols]

# print("Minimum block number:", transactions_df.block_number.min())
# print("Maximum block number:", transactions_df.block_number.max())

In [26]:
transactions_df = pd.read_csv(r'data/transactions_small.csv')
transactions_df.shape

(106896300, 10)

In [27]:
transactions_df.head()

,transaction_id,block_number,transaction_index,value,gas,gas_price,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,block_timestamp
0,133302900000000000,13330290,0,300000000000000000,611894,74759089945,7.500000e+10,1.500000e+09,2,1633049163
1,133302900000000001,13330290,1,0,184981,74759089945,1.770000e+11,1.500000e+09,2,1633049163
2,133302900000000002,13330290,2,0,278299,74259089945,7.500000e+10,1.000000e+09,2,1633049163
3,133302900000000003,13330290,3,74423577302830566,21000,1613659229046,NaN,NaN,0,1633049163
4,133302900000000004,13330290,4,9368049882777163,21000,111116168767,1.111162e+11,1.111162e+11,2,1633049163


In [28]:
print("Minimum block number:", transactions_df.block_number.min())
print("Maximum block number:", transactions_df.block_number.max())

Minimum block number: 12961718
Maximum block number: 13527858


In [29]:
print("# of unique blocks:", len(pd.unique(transactions_df.block_number)))

# of unique blocks: 556937


In [30]:
transactions_df.describe()

,transaction_id,block_number,transaction_index,gas,gas_price,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,block_timestamp
count,1.068963e+08,1.068963e+08,1.068963e+08,1.068963e+08,1.068963e+08,5.248754e+07,5.248754e+07,1.068963e+08,1.068963e+08
mean,1.324922e+17,1.324922e+07,1.479638e+02,1.734908e+05,9.889336e+10,1.595239e+11,1.693126e+10,9.823450e-01,1.631970e+09
std,1.655212e+15,1.655212e+05,1.314379e+02,3.818745e+05,5.156944e+12,1.598618e+13,7.357534e+12,9.996853e-01,2.221511e+06
min,1.296172e+17,1.296172e+07,0.000000e+00,2.100000e+04,0.000000e+00,1.423420e+09,0.000000e+00,0.000000e+00,1.628122e+09
25%,1.310515e+17,1.310514e+07,5.100000e+01,3.650700e+04,5.496497e+10,7.800000e+10,1.500000e+09,0.000000e+00,1.630036e+09
50%,1.324835e+17,1.324835e+07,1.170000e+02,9.926200e+04,7.802267e+10,1.114272e+11,1.940000e+09,0.000000e+00,1.631951e+09
75%,1.339462e+17,1.339462e+07,2.130000e+02,2.260900e+05,1.139259e+11,1.655751e+11,3.000000e+09,2.000000e+00,1.633919e+09
max,1.352786e+17,1.352786e+07,1.430000e+03,3.017625e+07,5.324367e+16,1.157740e+17,5.324361e+16,2.000000e+00,1.635725e+09


## Receipts

In [22]:
receipts_df = pd.read_csv(r'data/receipts_small.csv')
receipts_df.shape

(177191385, 7)

In [23]:
receipts_df.head()

,transaction_id,transaction_index,block_number,receipt_cumulative_gas_used,receipt_gas_used,receipt_status,receipt_effective_gas_price
0,130970770000000042,42,13097077,20269277,615110,1,699757885383
1,130792690000000094,94,13079269,4909942,105802,1,62815025192
2,130989030000000240,240,13098903,16465005,218423,1,69615073997
3,131641020000000106,106,13164102,7507934,21000,1,157010799148
4,131636110000000098,98,13163611,5929557,21000,1,99382274301


In [24]:
print("Minimum block number:", receipts_df.block_number.min())
print("Maximum block number:", receipts_df.block_number.max())

Minimum block number: 12961719
Maximum block number: 13538797


In [25]:
print("# of unique blocks:", len(pd.unique(receipts_df.block_number)))

# of unique blocks: 567717


In [31]:
receipts_df.describe()

,transaction_id,transaction_index,block_number,receipt_cumulative_gas_used,receipt_gas_used,receipt_status,receipt_effective_gas_price
count,1.771914e+08,1.771914e+08,1.771914e+08,1.771914e+08,1.771914e+08,1.771914e+08,1.771914e+08
mean,1.321286e+17,1.470498e+02,1.321286e+07,1.004285e+07,8.239654e+04,9.635640e-01,9.559985e+10
std,1.573253e+15,1.311281e+02,1.573253e+05,7.987629e+06,1.693320e+05,1.873725e-01,5.661691e+12
min,1.296172e+17,0.000000e+00,1.296172e+07,1.926000e+04,1.352400e+04,0.000000e+00,0.000000e+00
25%,1.308061e+17,5.100000e+01,1.308061e+07,3.310022e+06,2.100000e+04,1.000000e+00,5.200000e+10
50%,1.319993e+17,1.160000e+02,1.319993e+07,7.915357e+06,4.610900e+04,1.000000e+00,7.443028e+10
75%,1.332139e+17,2.110000e+02,1.332138e+07,1.534848e+07,9.237800e+04,1.000000e+00,1.087488e+11
max,1.353880e+17,1.430000e+03,1.353880e+07,3.029293e+07,3.000000e+07,1.000000e+00,5.324367e+16


In [32]:
receipts_df.rename(columns={'receipt_cumulative_gas_used': 'cumulative_gas_used',
                            'receipt_gas_used': 'gas_used',
                            'receipt_status': 'status',
                            'receipt_effective_gas_price': 'effective_gas_price'
                           }, inplace=True)

In [33]:
receipts_df.head()

,transaction_id,transaction_index,block_number,cumulative_gas_used,gas_used,status,effective_gas_price
0,130970770000000042,42,13097077,20269277,615110,1,699757885383
1,130792690000000094,94,13079269,4909942,105802,1,62815025192
2,130989030000000240,240,13098903,16465005,218423,1,69615073997
3,131641020000000106,106,13164102,7507934,21000,1,157010799148
4,131636110000000098,98,13163611,5929557,21000,1,99382274301


In [34]:
cols = ['transaction_id', 'block_number', 'cumulative_gas_used',
    'gas_used', 'status', 'effective_gas_price']
receipts_df = receipts_df[cols]

In [ ]:
# cols = [
#     'transaction_id', 'block_number', 'cumulative_gas_used',
#     'gas_used', 'status', 'effective_gas_price'
# ]

# # where block_number between 12967000 and 12968000

# receipts_df = pd.read_sql(f"SELECT {', '.join(cols)} FROM ethereumetl.receipts_small  ", conn)
# receipts_df.shape

## Merge Transactions and Receipts

In [35]:
transactions_receipts_df = transactions_df.merge(receipts_df,
                                             how='inner',
                                             left_on=['transaction_id', 'block_number'],
                                             right_on=['transaction_id', 'block_number'])
transactions_receipts_df.shape

(174852847, 14)

In [36]:
transactions_receipts_df.head()

,transaction_id,block_number,transaction_index,value,gas,gas_price,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,block_timestamp,cumulative_gas_used,gas_used,status,effective_gas_price
0,133302900000000000,13330290,0,300000000000000000,611894,74759089945,7.500000e+10,1.500000e+09,2,1633049163,611894,611894,1,74759089945
1,133302900000000001,13330290,1,0,184981,74759089945,1.770000e+11,1.500000e+09,2,1633049163,746250,134356,1,74759089945
2,133302900000000002,13330290,2,0,278299,74259089945,7.500000e+10,1.000000e+09,2,1633049163,922183,175933,1,74259089945
3,133302900000000003,13330290,3,74423577302830566,21000,1613659229046,NaN,NaN,0,1633049163,943183,21000,1,1613659229046
4,133302900000000004,13330290,4,9368049882777163,21000,111116168767,1.111162e+11,1.111162e+11,2,1633049163,964183,21000,1,111116168767


In [37]:
#transactions_receipts_df = transactions_receipts_df.reset_index()
transactions_receipts_df.columns

Index(['transaction_id', 'block_number', 'transaction_index', 'value', 'gas',
       'gas_price', 'max_fee_per_gas', 'max_priority_fee_per_gas',
       'transaction_type', 'block_timestamp', 'cumulative_gas_used',
       'gas_used', 'status', 'effective_gas_price'],
      dtype='object')

In [38]:
# Calculate aggregated variables at block level
transactions_receipts_agg_df = transactions_receipts_df[['block_number', 'gas', 'gas_price', 'gas_used', 'effective_gas_price']]\
        .groupby('block_number').agg(['min', 'mean', 'count'])
transactions_receipts_agg_df.columns = transactions_receipts_agg_df.columns.map('_'.join).str.strip('_')
transactions_receipts_agg_df

,gas_min,gas_mean,gas_count,gas_price_min,gas_price_mean,gas_price_count,gas_used_min,gas_used_mean,gas_used_count,effective_gas_price_min,effective_gas_price_mean,effective_gas_price_count
block_number,,,,,,,,,,,,
12961719,21000,173559.787234,470,0,6.130997e+10,470,19730,63727.651064,470,0,6.130997e+10,470
12961720,21000,255567.969388,392,0,5.420722e+10,392,19853,76436.489796,392,0,5.420722e+10,392
12961721,21000,223509.886598,388,0,5.656310e+10,388,19301,77255.876289,388,0,5.656310e+10,388
12961723,21000,171902.410714,336,39000000000,4.676679e+10,336,19393,89120.190476,336,39000000000,4.676679e+10,336
12961724,21000,242598.598039,408,0,5.865438e+10,408,16220,73487.240196,408,0,5.865438e+10,408
...,...,...,...,...,...,...,...,...,...,...,...,...
13527854,21000,138959.481081,185,172555259180,1.880757e+11,185,21000,76115.308108,185,172555259180,1.880757e+11,185
13527855,21000,157866.857494,407,171833195532,2.008356e+11,407,21000,73802.872236,407,171833195532,2.008356e+11,407
13527856,21000,167440.677419,186,192157722010,2.027112e+11,186,21000,107680.623656,186,192157722010,2.027112e+11,186


In [39]:
# Keep only certain columns
transactions_receipts_agg_df = transactions_receipts_agg_df[['gas_min', 'gas_mean', 'gas_price_min', 'gas_price_mean', 
                                                            'gas_used_min', 'gas_used_mean', 'effective_gas_price_min',
                                                             'effective_gas_price_mean', 'effective_gas_price_count']]
transactions_receipts_agg_df.rename(columns={'effective_gas_price_count': 'number_transactions_in_block'}, inplace=True)
transactions_receipts_agg_df        

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,gas_min,gas_mean,gas_price_min,gas_price_mean,gas_used_min,gas_used_mean,effective_gas_price_min,effective_gas_price_mean,number_transactions_in_block
block_number,,,,,,,,,
12961719,21000,173559.787234,0,6.130997e+10,19730,63727.651064,0,6.130997e+10,470
12961720,21000,255567.969388,0,5.420722e+10,19853,76436.489796,0,5.420722e+10,392
12961721,21000,223509.886598,0,5.656310e+10,19301,77255.876289,0,5.656310e+10,388
12961723,21000,171902.410714,39000000000,4.676679e+10,19393,89120.190476,39000000000,4.676679e+10,336
12961724,21000,242598.598039,0,5.865438e+10,16220,73487.240196,0,5.865438e+10,408
...,...,...,...,...,...,...,...,...,...
13527854,21000,138959.481081,172555259180,1.880757e+11,21000,76115.308108,172555259180,1.880757e+11,185
13527855,21000,157866.857494,171833195532,2.008356e+11,21000,73802.872236,171833195532,2.008356e+11,407
13527856,21000,167440.677419,192157722010,2.027112e+11,21000,107680.623656,192157722010,2.027112e+11,186


In [40]:
cols = ['gas_mean', 'gas_price_mean', 'gas_used_mean', 'effective_gas_price_mean', 'number_transactions_in_block']
for col in cols:
    # Last 5 blocks
    transactions_receipts_agg_df[col+'_pct_chg_last_5'] = transactions_receipts_agg_df[col]/transactions_receipts_agg_df[col].shift(5)-1
    # 25 blocks ago to 5 blocks ago percentage changes
    transactions_receipts_agg_df[col+'_pct_chg_last_25_to_5'] = transactions_receipts_agg_df[col].shift(5)/transactions_receipts_agg_df[col].shift(25)-1
    # 50 blocks ago to 5 blocks ago percentage changes
    transactions_receipts_agg_df[col+'_pct_chg_last_50_to_5'] = transactions_receipts_agg_df[col].shift(5)/transactions_receipts_agg_df[col].shift(50)-1
    # 100 blocks ago to 5 blocks ago percentage changes
    transactions_receipts_agg_df[col+'_pct_chg_last_100_to_5'] = transactions_receipts_agg_df[col].shift(5)/transactions_receipts_agg_df[col].shift(100)-1

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
transactions_receipts_agg_df.head()

,gas_min,gas_mean,gas_price_min,gas_price_mean,gas_used_min,gas_used_mean,effective_gas_price_min,effective_gas_price_mean,number_transactions_in_block,gas_mean_pct_chg_last_5,...,gas_used_mean_pct_chg_last_50_to_5,gas_used_mean_pct_chg_last_100_to_5,effective_gas_price_mean_pct_chg_last_5,effective_gas_price_mean_pct_chg_last_25_to_5,effective_gas_price_mean_pct_chg_last_50_to_5,effective_gas_price_mean_pct_chg_last_100_to_5,number_transactions_in_block_pct_chg_last_5,number_transactions_in_block_pct_chg_last_25_to_5,number_transactions_in_block_pct_chg_last_50_to_5,number_transactions_in_block_pct_chg_last_100_to_5
block_number,,,,,,,,,,,,,,,,,,,,,
12961719,21000,173559.787234,0,6.130997e+10,19730,63727.651064,0,6.130997e+10,470,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961720,21000,255567.969388,0,5.420722e+10,19853,76436.489796,0,5.420722e+10,392,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961721,21000,223509.886598,0,5.656310e+10,19301,77255.876289,0,5.656310e+10,388,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961723,21000,171902.410714,39000000000,4.676679e+10,19393,89120.190476,39000000000,4.676679e+10,336,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12961724,21000,242598.598039,0,5.865438e+10,16220,73487.240196,0,5.865438e+10,408,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Point In Time

In [ ]:
# cols = [
#     'value', 'transaction_hash', 'log_index', 'block_number'
# ]

pit_df_old = pd.read_sql(f"SELECT * FROM ethereumetl.latest_avail_block_60 where number between 12967000 and 12968000 ", conn)
pit_df_old.shape

In [ ]:
pit_df_old.head()

In [42]:
pit_df = pd.read_csv(r'data/pit_60.csv')

In [43]:
pit_df.shape

(566141, 4)

In [44]:
pit_df.head()

,number,datetime,lag_cutoff,latest_avail_block
0,12961718,2021-08-05 00:00:04,2021-08-04 23:59:04,NaN
1,12961719,2021-08-05 00:00:07,2021-08-04 23:59:07,NaN
2,12961720,2021-08-05 00:00:47,2021-08-04 23:59:47,NaN
3,12961721,2021-08-05 00:01:03,2021-08-05 00:00:03,NaN
4,12961722,2021-08-05 00:01:25,2021-08-05 00:00:25,12961719.0


In [45]:
pit_df = pit_df.set_index('number')
pit_df.rename(columns={'lag_cutoff': 'lag_cutoff_60',
                       'latest_avail_block': 'latest_avail_60'},
             inplace=True)

In [ ]:
# pit_df = pit_df.set_index('number')
# pit_df['latest_avail_time_dt'] = pd.to_datetime(pit_df['latest_avail_time'], unit='s')
# pit_df.rename(columns={'lag_cutoff': 'lag_cutoff_60',
#                        'latest_avail_block': 'latest_avail_60',
#                       'latest_avail_time': 'latest_avail_time_60',
#                       'latest_avail_time_dt': 'latest_avail_time_dt_60'},
#              inplace=True)

In [ ]:
# pit_df = pit_df.set_index('number')
# pit_df['latest_avail_time_dt'] = pd.to_datetime(pit_df['latest_avail_time'], unit='s')
# pit_df.rename(columns={'lag_cutoff': 'lag_cutoff_60',
#                        'latest_avail': 'latest_avail_60',
#                       'latest_avail_time': 'latest_avail_time_60',
#                       'latest_avail_time_dt': 'latest_avail_time_dt_60'},
#              inplace=True)

In [46]:
pit_df.head()

,datetime,lag_cutoff_60,latest_avail_60
number,,,
12961718,2021-08-05 00:00:04,2021-08-04 23:59:04,NaN
12961719,2021-08-05 00:00:07,2021-08-04 23:59:07,NaN
12961720,2021-08-05 00:00:47,2021-08-04 23:59:47,NaN
12961721,2021-08-05 00:01:03,2021-08-05 00:00:03,NaN
12961722,2021-08-05 00:01:25,2021-08-05 00:00:25,12961719.0


# Merge Data Sets

In [47]:
print("blocks length:", len(blocks_df))
print("pit length:", len(pit_df))
print("transactions length:", len(transactions_df))
print("receipts length:", len(receipts_df))
print("transactions_receipts_agg_df length:", len(transactions_receipts_agg_df))

blocks length: 566141
pit length: 566141
transactions length: 106896300
receipts length: 177191385
transactions_receipts_agg_df length: 556936


In [48]:
# Merge point-in-time with blocks
# merged_df = pit_df[['lag_cutoff_60', 'latest_avail_60', 'latest_avail_time_60', 'latest_avail_time_dt_60']].merge(blocks_df,
#                         how='inner',
#                         left_index=True,
#                         right_index=True)

# Merge point-in-time with blocks
merged_df = pit_df[['lag_cutoff_60', 'latest_avail_60', 'datetime']].merge(blocks_df,
                        how='inner',
                        left_index=True,
                        right_index=True)

merged_df.shape

(566141, 37)

In [49]:
print("Length of merged_df", len(merged_df))
merged_df.tail()

Length of merged_df 566141


,lag_cutoff_60,latest_avail_60,datetime,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,...,size_pct_chg_last_50_to_5,size_pct_chg_last_100_to_5,gas_limit_pct_chg_last_5,gas_limit_pct_chg_last_25_to_5,gas_limit_pct_chg_last_50_to_5,gas_limit_pct_chg_last_100_to_5,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_25_to_5,base_fee_per_gas_pct_chg_last_50_to_5,base_fee_per_gas_pct_chg_last_100_to_5
number,,,,,,,,,,,,,,,,,,,,,
13527854,2021-10-31 23:57:12,13527848.0,2021-10-31 23:58:12,10142061783382444,3.349723e+22,59466,30029295,14081332,1635724692,185,...,0.240670,-0.881631,0.001617,-0.000640,-0.000640,-0.000640,-0.202803,0.322248,-0.527752,0.701928
13527855,2021-10-31 23:57:28,13527849.0,2021-10-31 23:58:28,10142130502859180,3.349724e+22,113946,30058619,30037769,1635724708,407,...,33.849049,0.481497,0.002595,-0.000639,0.000000,-0.000639,-0.126427,0.064287,-0.549740,0.369973
13527856,2021-10-31 23:58:08,13527852.0,2021-10-31 23:59:08,10127342585857119,3.349725e+22,101346,30029266,20028596,1635724748,186,...,-0.704577,0.736733,0.000976,0.000639,0.000639,0.000639,-0.037228,-0.034343,-0.497471,0.429607
13527857,2021-10-31 23:58:18,13527854.0,2021-10-31 23:59:18,10127411305333855,3.349726e+22,12068,30000000,2550891,1635724758,30,...,6.175762,0.080803,-0.000976,0.000639,0.001616,0.000976,0.127682,-0.235097,-0.593387,0.441729
13527858,2021-10-31 23:58:20,13527854.0,2021-10-31 23:59:20,10132425049862023,3.349727e+22,132704,29980831,29719237,1635724760,408,...,2.207616,-0.644881,-0.000639,0.000000,0.000639,0.000639,-0.061103,-0.144650,-0.512152,0.476677


In [50]:
# Merge point-in-time/blocks with transactions_receipts_agg_df
pipeline_df = merged_df.merge(transactions_receipts_agg_df,
                        how='left',
                        left_index=True,
                        right_index=True)
pipeline_df.shape

(566141, 66)

In [51]:
pipeline_df.columns

Index(['lag_cutoff_60', 'latest_avail_60', 'datetime', 'difficulty',
       'total_difficulty', 'size', 'gas_limit', 'gas_used', 'timestamp',
       'transaction_count', 'base_fee_per_gas', 'timestamp_2',
       'difficulty_pct_chg', 'total_difficulty_pct_chg', 'size_pct_chg',
       'gas_limit_pct_chg', 'base_fee_per_gas_pct_chg',
       'difficulty_pct_chg_last_5', 'difficulty_pct_chg_last_25_to_5',
       'difficulty_pct_chg_last_50_to_5', 'difficulty_pct_chg_last_100_to_5',
       'total_difficulty_pct_chg_last_5',
       'total_difficulty_pct_chg_last_25_to_5',
       'total_difficulty_pct_chg_last_50_to_5',
       'total_difficulty_pct_chg_last_100_to_5', 'size_pct_chg_last_5',
       'size_pct_chg_last_25_to_5', 'size_pct_chg_last_50_to_5',
       'size_pct_chg_last_100_to_5', 'gas_limit_pct_chg_last_5',
       'gas_limit_pct_chg_last_25_to_5', 'gas_limit_pct_chg_last_50_to_5',
       'gas_limit_pct_chg_last_100_to_5', 'base_fee_per_gas_pct_chg_last_5',
       'base_fee_per_gas_

In [52]:
pd.options.display.max_columns = None
pipeline_df.describe()

,latest_avail_60,difficulty,total_difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,difficulty_pct_chg,total_difficulty_pct_chg,size_pct_chg,gas_limit_pct_chg,base_fee_per_gas_pct_chg,difficulty_pct_chg_last_5,difficulty_pct_chg_last_25_to_5,difficulty_pct_chg_last_50_to_5,difficulty_pct_chg_last_100_to_5,total_difficulty_pct_chg_last_5,total_difficulty_pct_chg_last_25_to_5,total_difficulty_pct_chg_last_50_to_5,total_difficulty_pct_chg_last_100_to_5,size_pct_chg_last_5,size_pct_chg_last_25_to_5,size_pct_chg_last_50_to_5,size_pct_chg_last_100_to_5,gas_limit_pct_chg_last_5,gas_limit_pct_chg_last_25_to_5,gas_limit_pct_chg_last_50_to_5,gas_limit_pct_chg_last_100_to_5,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_25_to_5,base_fee_per_gas_pct_chg_last_50_to_5,base_fee_per_gas_pct_chg_last_100_to_5,gas_min,gas_mean,gas_price_min,gas_price_mean,gas_used_min,gas_used_mean,effective_gas_price_min,effective_gas_price_mean,number_transactions_in_block,gas_mean_pct_chg_last_5,gas_mean_pct_chg_last_25_to_5,gas_mean_pct_chg_last_50_to_5,gas_mean_pct_chg_last_100_to_5,gas_price_mean_pct_chg_last_5,gas_price_mean_pct_chg_last_25_to_5,gas_price_mean_pct_chg_last_50_to_5,gas_price_mean_pct_chg_last_100_to_5,gas_used_mean_pct_chg_last_5,gas_used_mean_pct_chg_last_25_to_5,gas_used_mean_pct_chg_last_50_to_5,gas_used_mean_pct_chg_last_100_to_5,effective_gas_price_mean_pct_chg_last_5,effective_gas_price_mean_pct_chg_last_25_to_5,effective_gas_price_mean_pct_chg_last_50_to_5,effective_gas_price_mean_pct_chg_last_100_to_5,number_transactions_in_block_pct_chg_last_5,number_transactions_in_block_pct_chg_last_25_to_5,number_transactions_in_block_pct_chg_last_50_to_5,number_transactions_in_block_pct_chg_last_100_to_5
count,5.661370e+05,5.661410e+05,5.661410e+05,5.661410e+05,5.661410e+05,5.661410e+05,5.661410e+05,566141.000000,5.628590e+05,5.661400e+05,5.661400e+05,566140.000000,566140.000000,562858.000000,566136.000000,566116.000000,566091.000000,566041.000000,5.661360e+05,5.661160e+05,5.660910e+05,566041.000000,566136.000000,566116.000000,566091.000000,566041.000000,566136.000000,566116.000000,566091.000000,566041.000000,562854.000000,562834.000000,562809.000000,562759.000000,5.569360e+05,5.569360e+05,5.569360e+05,5.569360e+05,5.569360e+05,5.569360e+05,5.569360e+05,5.569360e+05,556936.000000,556931.000000,556911.000000,556886.000000,556836.000000,5.569310e+05,5.569110e+05,5.568860e+05,5.568360e+05,556931.000000,556911.000000,556886.000000,556836.000000,5.569310e+05,5.569110e+05,5.568860e+05,5.568360e+05,556931.000000,556911.000000,556886.000000,556836.000000
mean,1.324478e+07,8.881305e+15,3.086043e+22,7.718583e+04,2.992329e+07,1.544216e+07,1.631910e+09,188.815684,8.389799e+10,7.155172e-07,2.872804e-07,5.531328,0.000002,0.003672,0.000004,0.000014,0.000032,0.000068,1.436403e-06,5.745620e-06,1.292769e-05,0.000027,4.114598,4.054488,4.088010,4.129498,0.000010,0.000036,0.000080,0.000167,0.012018,0.031181,0.061357,0.086395,2.166753e+04,1.889297e+05,8.506328e+10,1.022344e+11,2.134077e+04,8.907296e+04,8.506328e+10,1.022344e+11,313.955009,0.125859,0.133857,0.138133,0.142811,inf,inf,inf,inf,0.104803,0.117146,0.123761,0.127360,inf,inf,inf,inf,1.718159,1.703634,1.717208,1.731129
std,1.634298e+05,7.772472e+14,1.456622e+21,5.759466e+04,1.146282e+06,1.029474e+07,2.193156e+06,142.875043,7.564743e+10,7.033183e-04,1.283190e-08,30.235284,0.001540,0.086000,0.001557,0.003085,0.004593,0.006636,6.415319e-08,2.565340e-07,5.769133e-07,0.000001,23.649602,23.228035,23.283212,23.399238,0.003215,0.006055,0.008896,0.012699,0.160737,0.333681,0.758804,0.915449,7.859786e+04,1.995758e+05,9.382589e+10,2.328123e+11,7.454271e+04,9.926500e+04,9.382589e+10,2.328123e+11,254.926877,1.206740,1.265515,1.248601,1.262789,NaN,NaN,NaN,NaN,1.144399,1.219309,1.305462,1.260407,NaN,NaN,NaN,NaN,8.335019,8.288193,8.585240,8.485832
min,1.296172e+07,7.213602e+15,2.846921e+22,5.200000e+02,1.489037e+07,0.000000e+00,1.628122e+09,0.000000,1.000000e+09,-8.297229e-03,2.5

In [53]:
pipeline_df.index

Int64Index([12961718, 12961719, 12961720, 12961721, 12961722, 12961723,
            12961724, 12961725, 12961726, 12961727,
            ...
            13527849, 13527850, 13527851, 13527852, 13527853, 13527854,
            13527855, 13527856, 13527857, 13527858],
           dtype='int64', name='number', length=566141)

# Save the final data set

In [54]:
from io import StringIO 
import boto3

bucket = 'sagemaker-w210-eth' # already created on S3
csv_buffer = StringIO()
pipeline_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'pipeline_df_20211113_2.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'C0P4XA3PYEY1WE7G',
  'HostId': 'uzTd2oQWvb73tnUSFCb519kFFsWb/3eHrYgqbv9ts9k3cXQDqJNF5l99fvBwZaF6kfBK7IUtEpc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'uzTd2oQWvb73tnUSFCb519kFFsWb/3eHrYgqbv9ts9k3cXQDqJNF5l99fvBwZaF6kfBK7IUtEpc=',
   'x-amz-request-id': 'C0P4XA3PYEY1WE7G',
   'date': 'Sun, 14 Nov 2021 01:28:08 GMT',
   'etag': '"230cd6e1e81d89f819b19406d8f2dece"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"230cd6e1e81d89f819b19406d8f2dece"'}